In [3]:
import pandas as pd
import pickle

In [4]:
def make_fset(x, drop=True):
    if len(set(x.split(' '))) < 2:
        print(f"WARNING: Features for '{x}-{x}' (self-self PPI) detected ...")
        if drop == False:
            x1 = x.split(' ')[0]
            fset = frozenset({x1,x1})
            return(fset)
        else:
            return(None)
    else:
        x1 = x.split(' ')[0]
        x2 = x.split(' ')[1]
        fset = frozenset({x1,x2})
        return(fset)
    
def map_phyla(ppi, ppi_dict):
    phyla = ppi_dict[ppi]['phyla']
    n_phyla = ppi_dict[ppi]['n_phyla']
    return(phyla, n_phyla)

In [5]:
fmat_file_dir = '../ppi_ml/data/features/pkl_files'
score_file = '../ppi_ml/results/ppi_predict/scored_interactions_all_LinearSVC.csv'
outfile = '../ppi_ml/results/ppi_predict/scored_interactions_all_LinearSVC_phyla.csv'

In [6]:
ppi_scores = pd.read_csv(score_file)
ppi_scores['fs'] = [make_fset(i) for i in ppi_scores['ID']]
ppi_scores

,ID,label,ppi_score,set,fs
0,KOG0357 KOG0363,1,1.000000,train,"(KOG0357, KOG0363)"
1,KOG0361 KOG0357,1,1.000000,train,"(KOG0357, KOG0361)"
2,KOG0358 KOG0357,1,1.000000,train,"(KOG0357, KOG0358)"
3,KOG0359 KOG0363,1,1.000000,train,"(KOG0363, KOG0359)"
4,KOG0364 KOG0357,1,1.000000,train,"(KOG0357, KOG0364)"
...,...,...,...,...,...
4491713,ENOG502QPP0 KOG1073,0,0.046646,predict,"(KOG1073, ENOG502QPP0)"
4491714,ENOG502QPP0 ENOG502QWQ0,0,0.046267,predict,"(ENOG502QWQ0, ENOG502QPP0)"
4491715,KOG1084 KOG4735,0,0.045761,predict,"(KOG4735, KOG1084)"
4491716,KOG2106 ENOG502QWQ0,0,0.045250,predict,"(KOG2106, ENOG502QWQ0)"


# had to redo this due to something weird happening with the first try where ppi clades were incorrectly assigned 

### old code:

In [5]:
# def map_phyla(ppi):
#     phyla = []
#     if ppi in amor_ppis:
#         phyla.append('Amorphea')
#     if ppi in exca_ppis:
#         phyla.append('Excavata')
#     if ppi in tsar_ppis:
#         phyla.append('TSAR')
#     if ppi in viri_ppis:
#         phyla.append('Archaeplastida')
#     n = len(phyla)
#     sort(phyla)
#     phyla_str = ', '.join(phyla)
#     return(n, phyla_str)

# amor_file = f'{fmat_file_dir}/featmat_animals.pkl'
# exca_file = f'{fmat_file_dir}/featmat_excavate.pkl'
# tsar_file = f'{fmat_file_dir}/featmat_tsar.pkl'
# viri_file = f'{fmat_file_dir}/featmat_plants.pkl'
# humap_file = f'{fmat_file_dir}/featmat_humap.pkl'

# print(f'Getting Excavata PPIs ...')
# exca = pd.read_pickle(exca_file)
# exca_ppis = set([make_fset(i) for i in exca['ID']])
# del exca

# print(f'Getting TSAR PPIs ...')
# tsar = pd.read_pickle(tsar_file)
# tsar_ppis = set([make_fset(i) for i in tsar['ID']])
# del tsar

# print(f'Getting Archaeplastida PPIs ...')
# viri = pd.read_pickle(viri_file)
# viri_ppis = set([make_fset(i) for i in viri['ID']])
# del viri

# print(f'Getting Amorphea PPIs ...')
# amor = pd.read_pickle(amor_file)
# amor_ppis = set([make_fset(i) for i in amor['ID']])
# del amor

# print(f'Getting huMAP PPIs ...')
# hmap = pd.read_pickle(humap_file)
# hmap_ppis = set([make_fset(i) for i in hmap['ID']])
# del hmap

# print(f'Combining Amorphea and huMAP PPIs ...')
# amor_ppis.update(hmap_ppis)

# print(f'Mapping PPIs to phyla ...')
# ppi_scores[['n_phyla', 'phyla']] = [map_phyla(i) for i in ppi_scores['fs']]
# ppi_scores.drop(['fs'], axis=1, inplace=True)
# ppi_scores.to_csv(outfile, index=False)

Getting Excavata PPIs ...
Getting TSAR PPIs ...
Getting Archaeplastida PPIs ...
Getting Amorphea PPIs ...
Getting huMAP PPIs ...
Combining Amorphea and huMAP PPIs ...


# debugged mapping

In [7]:
import re
from tqdm import tqdm

In [8]:
# feature matrix
fmat_file = f'{fmat_file_dir}/featmat_allexps_p3c2.pkl'
fmat = pd.read_pickle(fmat_file)
fmat['fs'] = [make_fset(i) for i in fmat['ID']]

In [86]:
# species cod../ppi_ml/p
species_file = '../ppi_ml/data/meta/speciesinfo_clades.csv'
spec_df = pd.read_csv(species_file)
spec_df['code'] = [i.lower() for i in spec_df['code']]
spec_df['clade'] = ['Excavata' if i == 'Excavate' else i for i in spec_df['clade']]
clade_dict = dict(zip(spec_df['code'], spec_df['clade']))
clade_dict

{'arath': 'Archaeplastida',
 'braol': 'Archaeplastida',
 'brart': 'Amorphea',
 'caeel': 'Amorphea',
 'camnx': 'Archaea',
 'cansa': 'Archaeplastida',
 'cerri': 'Archaeplastida',
 'cheqi': 'Archaeplastida',
 'chlre': 'Archaeplastida',
 'cocnu': 'Archaeplastida',
 'desvh': 'Bacteria',
 'dicdi': 'Amorphea',
 'drome': 'Amorphea',
 'ecoli': 'Bacteria',
 'euggr': 'Excavata',
 'halsa': 'Archaea',
 'human': 'Amorphea',
 'maize': 'Archaeplastida',
 'mouse': 'Amorphea',
 'nemve': 'Amorphea',
 'orysj': 'Archaeplastida',
 'phatc': 'TSAR',
 'pig': 'Amorphea',
 'plaba': 'TSAR',
 'plaf7': 'TSAR',
 'plakh': 'TSAR',
 'pseae': 'Bacteria',
 'selml': 'Archaeplastida',
 'sollc': 'Archaeplastida',
 'soybn': 'Archaeplastida',
 'staa8': 'Bacteria',
 'strpu': 'Amorphea',
 'tetts': 'TSAR',
 'tryb2': 'Excavata',
 'wheat': 'Archaeplastida',
 'xenla': 'Amorphea',
 'yeast': 'Amorphea'}

In [93]:
###########################
# parallelized this in a script:
# map_ppis2clades.py
###########################

# map ppis to phyla
# phyla_dict = dict()
# for i in tqdm(range(len((fmat)))):
#     ppi_id = fmat['fs'][i]
#     # get non-zero features
#     feats = list(fmat[fmat['fs']==ppi_id].dropna(axis=1,how='all'))[1:]
#     species_set = set()
#     clade_set = set()
#     # get species names from features
#     for f in feats:
#         code = f.split('.', 1)[0]
#         if code != 'fs':
#             species_set.add(code)
#             clade_set.add(clade_dict[code])
#     clades_sorted = sorted(list(clade_set))
#     if 'Archaeplastida' in clade_set:
#         clades_sorted.remove('Archaeplastida')
#         clades_sorted.append('Archaeplastida')
#     if ppi_id not in phyla_dict.keys():
#         phyla_dict[ppi_id] = {'n_phyla':{}, 'phyla':{}}
#         phyla_dict[ppi_id]['n_phyla'] = len(clade_set)
#         phyla_dict[ppi_id]['phyla'] = ', '.join(clades_sorted)

  0%|                              | 1405/4491718 [06:54<367:34:07,  3.39it/s]


KeyboardInterrupt: 

# oof, that takes way too long
# going to split the fmat into chunks and process in parallel to make the dictionary

In [123]:
import numpy as np

In [97]:
def find_factors(n):
    factors = []
    for i in range(1, n + 1):
        if n % i == 0:
            factors.append(i)
    return factors

In [114]:
###
# [code below executed successfully]
###

# factors = find_factors(len(fmat))
# options = [x for x in factors if 2 < x <= 48]
# chunk_num = options[-1]
# chunks = np.split(fmat, chunk_num)

# split_dir = '../ppi_ml/data/map_phyla/split_fmat'
# for i, df in enumerate(chunks):
#     df_out = df.reset_index(drop=True)
#     df_out.to_pickle(f'{split_dir}/fmat_{i}.pkl')

In [15]:
# test_df = fmat.head(25).reset_index(drop=True)
# test_df.to_pickle('../ppi_ml/data/map_phyla/split_fmat/fmat_test.pkl')
# with open('../ppi_ml/data/map_phyla/split_phyla_dict/phyla_dict_test.pkl', 'rb') as f:
#     test_dict = pickle.load(f)
# test_dict

{frozenset({'ENOG502QPIC', 'ENOG502QR7H'}): {'n_phyla': 2,
  'phyla': 'TSAR, Archaeplastida'},
 frozenset({'ENOG502QPIC', 'KOG0253'}): {'n_phyla': 2,
  'phyla': 'TSAR, Archaeplastida'},
 frozenset({'ENOG502QPIC', 'KOG0734'}): {'n_phyla': 2,
  'phyla': 'TSAR, Archaeplastida'},
 frozenset({'ENOG502QPIC', 'KOG0739'}): {'n_phyla': 2,
  'phyla': 'TSAR, Archaeplastida'},
 frozenset({'ENOG502QPIC', 'KOG0935'}): {'n_phyla': 2,
  'phyla': 'TSAR, Archaeplastida'},
 frozenset({'ENOG502QPIC', 'KOG1440'}): {'n_phyla': 2,
  'phyla': 'TSAR, Archaeplastida'},
 frozenset({'ENOG502QPIC', 'KOG1723'}): {'n_phyla': 2,
  'phyla': 'TSAR, Archaeplastida'},
 frozenset({'ENOG502QPIC', 'KOG1805'}): {'n_phyla': 2,
  'phyla': 'TSAR, Archaeplastida'},
 frozenset({'ENOG502QPIC', 'KOG1865'}): {'n_phyla': 2,
  'phyla': 'TSAR, Archaeplastida'},
 frozenset({'ENOG502QPIC', 'KOG1949'}): {'n_phyla': 2,
  'phyla': 'TSAR, Archaeplastida'},
 frozenset({'ENOG502QPIC', 'KOG1956'}): {'n_phyla': 2,
  'phyla': 'TSAR, Archaeplastid

In [124]:
!for i in {0..21}; do echo "python3 scripts/map_ppis2clades.py ppi_ml/data/map_phyla/split_fmat/fmat_${i}.pkl ppi_ml/data/map_phyla/split_phyla_dict/phyla_dict_${i}.pkl"; done

python3 scripts/map_ppis2clades.py ppi_ml/data/map_phyla/split_fmat/fmat_0.pkl ppi_ml/data/map_phyla/split_phyla_dict/phyla_dict_0.pkl
python3 scripts/map_ppis2clades.py ppi_ml/data/map_phyla/split_fmat/fmat_1.pkl ppi_ml/data/map_phyla/split_phyla_dict/phyla_dict_1.pkl
python3 scripts/map_ppis2clades.py ppi_ml/data/map_phyla/split_fmat/fmat_2.pkl ppi_ml/data/map_phyla/split_phyla_dict/phyla_dict_2.pkl
python3 scripts/map_ppis2clades.py ppi_ml/data/map_phyla/split_fmat/fmat_3.pkl ppi_ml/data/map_phyla/split_phyla_dict/phyla_dict_3.pkl
python3 scripts/map_ppis2clades.py ppi_ml/data/map_phyla/split_fmat/fmat_4.pkl ppi_ml/data/map_phyla/split_phyla_dict/phyla_dict_4.pkl
python3 scripts/map_ppis2clades.py ppi_ml/data/map_phyla/split_fmat/fmat_5.pkl ppi_ml/data/map_phyla/split_phyla_dict/phyla_dict_5.pkl
python3 scripts/map_ppis2clades.py ppi_ml/data/map_phyla/split_fmat/fmat_6.pkl ppi_ml/data/map_phyla/split_phyla_dict/phyla_dict_6.pkl
python3 scripts/map_ppis2clades.py ppi_ml/data/map_phyl

In [17]:
# merge dictionaries that were output by scripts
dict_dir = '../ppi_ml/data/map_phyla/split_phyla_dict'
merged = dict()
for i in range(0,22):
    with open(f'{dict_dir}/phyla_dict_{i}.pkl', 'rb') as f:
        dict_part = pickle.load(f)
    merged.update(dict_part)
print(f'# ppis in final dictionary: {len(merged)}')
with open('../ppi_ml/data/map_phyla/phyla_dict.pkl', 'wb') as f:
    if len(merged) == len(ppi_scores):
        pickle.dump(merged, f)

# ppis in final dictionary: 4491718


# execute mapping (finally)

In [19]:
with open('../ppi_ml/data/map_phyla/phyla_dict.pkl', 'rb') as f:
    phyla_dict = pickle.load(f)
ppi_scores[['phyla','n_phyla']] = [map_phyla(i, phyla_dict) for i in ppi_scores['fs']]
ppi_scores

,ID,label,ppi_score,set,fs,phyla,n_phyla
0,KOG0357 KOG0363,1,1.000000,train,"(KOG0357, KOG0363)","Amorphea, Excavata, TSAR, Archaeplastida",4
1,KOG0361 KOG0357,1,1.000000,train,"(KOG0357, KOG0361)","Amorphea, Excavata, TSAR, Archaeplastida",4
2,KOG0358 KOG0357,1,1.000000,train,"(KOG0357, KOG0358)","Amorphea, Excavata, TSAR, Archaeplastida",4
3,KOG0359 KOG0363,1,1.000000,train,"(KOG0363, KOG0359)","Amorphea, Excavata, TSAR, Archaeplastida",4
4,KOG0364 KOG0357,1,1.000000,train,"(KOG0357, KOG0364)","Amorphea, Excavata, TSAR, Archaeplastida",4
...,...,...,...,...,...,...,...
4491713,ENOG502QPP0 KOG1073,0,0.046646,predict,"(KOG1073, ENOG502QPP0)","Amorphea, Excavata",2
4491714,ENOG502QPP0 ENOG502QWQ0,0,0.046267,predict,"(ENOG502QWQ0, ENOG502QPP0)","Amorphea, Excavata",2
4491715,KOG1084 KOG4735,0,0.045761,predict,"(KOG4735, KOG1084)","Amorphea, Archaeplastida",2
4491716,KOG2106 ENOG502QWQ0,0,0.045250,predict,"(KOG2106, ENOG502QWQ0)","Amorphea, Excavata",2


In [23]:
ppi_scores[ppi_scores.n_phyla == 4]

,ID,label,ppi_score,set,phyla,n_phyla
0,KOG0357 KOG0363,1,1.000000,train,"Amorphea, Excavata, TSAR, Archaeplastida",4
1,KOG0361 KOG0357,1,1.000000,train,"Amorphea, Excavata, TSAR, Archaeplastida",4
2,KOG0358 KOG0357,1,1.000000,train,"Amorphea, Excavata, TSAR, Archaeplastida",4
3,KOG0359 KOG0363,1,1.000000,train,"Amorphea, Excavata, TSAR, Archaeplastida",4
4,KOG0364 KOG0357,1,1.000000,train,"Amorphea, Excavata, TSAR, Archaeplastida",4
...,...,...,...,...,...,...
4491695,KOG4210 KOG0069,0,0.049650,predict,"Amorphea, Excavata, TSAR, Archaeplastida",4
4491697,KOG0888 KOG0898,0,0.049538,predict,"Amorphea, Excavata, TSAR, Archaeplastida",4
4491698,KOG3627 KOG0023,0,0.049181,predict,"Amorphea, Excavata, TSAR, Archaeplastida",4
4491699,KOG2573 KOG3367,0,0.049178,predict,"Amorphea, Excavata, TSAR, Archaeplastida",4


In [22]:
ppi_scores.drop(['fs'], axis=1, inplace=True)
ppi_scores.to_csv(outfile, index=False)

'../ppi_ml/results/ppi_predict/scored_interactions_all_LinearSVC_phyla.csv'

# troubleshooting code below here

In [32]:
# number bugged ppis
print(ppi_scores[ppi_scores.n_phyla < 2].groupby('phyla').size())

# all bugged ppis
bug_df = ppi_scores[ppi_scores.n_phyla < 2]
bugged_ppis = set([make_fset(i) for i in bug_df['ID']])
print(f'\ntotal # bugged = {len(bugged_ppis)}')

phyla
                      6
Amorphea          15395
Archaeplastida     6330
Excavata             21
TSAR                771
dtype: int64

total # bugged = 22523


In [81]:
# feats for all bugged ppis
bugged_feats = fmat[fmat['fs'].isin(bugged_ppis)].reset_index(drop=True)
print(f'total # bugged found in feature matrix  = {len(bugged_feats)}')

total # bugged found in feature matrix  = 22523


In [83]:
debugged = dict()
for i in range(len((bugged_feats))):
    ppi_id = bugged_feats['fs'][i]
    # get non-zero features
    feats = list(bugged_feats[bugged_feats['fs']==ppi_id].dropna(axis=1,how='all'))[1:]
    species_set = set()
    clade_set = set()
    # get species names from features
    for f in feats:
        code = f.split('.', 1)[0]
        if code != 'fs':
            species_set.add(code)
            clade_set.add(clade_dict[code])
    if ppi_id not in debugged.keys():
        debugged[ppi_id] = {'n_phyla':{}, 'phyla':{}}
        debugged[ppi_id]['n_phyla'] = len(clade_set)
        debugged[ppi_id]['phyla'] = ', '.join(list(clade_set))

In [79]:
debugged[ppi_id]

{'n_phyla': 2, 'phyla': 'Excavata, Archaeplastida'}

In [ ]:
feature_file = '../ppi_ml/data/all_cfms_features.txt'
feature_cols = [line.strip() for line in open(feature_file, 'r')]
debug_elut = ',,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0.643846561005284,4.32261802590737,0.375502524869173,0.336512015690195,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0.815652468678013,5.08746717749884,0.307858483989912,0.183222015248467,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,'
debug_elut_lst = debug_elut.split(',')
non_empty = [index for index, value in enumerate(debug_elut_lst) if value]
[feature_cols[i] for i in non_empty]

TypeError: list indices must be integers or slices, not list